<a href="https://colab.research.google.com/github/YogiJungHoonYeon/Artificial-Intelligence-Lang/blob/main/Audio_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pip

In [ ]:
!pip install pydub
!pip install SpeechRecognition

#Module

In [ ]:
import os
import IPython.display as ipd
import speech_recognition as sr
import io
from google.colab import drive
import shutil
import re

#Drive Mount

In [ ]:
def drive_mount(drive_name):
  dir_name = os.getcwd()
  drive_path = os.path.join(dir_name,drive_name)
  drive.mount(drive_path)
drive_mount('drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#mkdir

In [ ]:
os.mkdir('wav_folder')

#Datasets

In [ ]:
from pydub import AudioSegment, silence,effects
silence_threshold = -35
min_silence_duration = 500
def split_audio_by_silence(input_file, output_dir, silence_threshold, min_silence_duration):
    audio = AudioSegment.from_file(input_file)
    audio = audio + 5
    audio = effects.low_pass_filter(audio, 3000)  # 높은 주파수 노이즈 감소
    audio = effects.high_pass_filter(audio, 30)  # 낮은 주파수 노이즈 감소
    audio = effects.normalize(audio)


    chunks = silence.split_on_silence(
        audio,
        min_silence_len=min_silence_duration,
        silence_thresh=silence_threshold
    )

    for i, chunk in enumerate(chunks):
        output_file = f"{output_dir}/chunk_{i}.wav"
        chunk.export(output_file, format="wav")
    print('chunk 생성 끝')


In [ ]:
def make_dataset(wav_dir):
  recognizer = sr.Recognizer()
  for file_name in os.listdir(wav_dir):
    file_path = os.path.join(wav_dir,file_name)
    if os.path.isdir(file_path) != True:
      with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
      try:
        text = recognizer.recognize_google(audio, language="ko-KR")
      except sr.UnknownValueError:
        text = "음성을 인식할 수 없음"
      new_filename = f"{text}.wav"
      #print(new_filename)
      new_file_path = os.path.join(wav_dir, new_filename)
      os.rename(file_path, new_file_path)
  print('recognize 끝')

In [ ]:
class dataset_generate():
  def __init__(self,input_file,input_folder,wav_dir,silence_threshold,min_silence_duration,choice = True):
    self.input_file = input_file
    self.input_folder = input_folder
    self.wav_dir = wav_dir
    self.silence_threshold = silence_threshold
    self.min_silence_duration = min_silence_duration
    self.choice = choice
  def call(self):
    num_re = re.compile('[0-9a-zA-Z]')
    if self.choice == True:
      if os.path.isdir(self.wav_dir) == False:
        os.mkdir(self.wav_dir)
      split_audio_by_silence(self.input_file, self.wav_dir, self.silence_threshold, self.min_silence_duration)

      make_dataset(self.wav_dir)

      for file_name in os.listdir(self.wav_dir):
        if file_name == '음성을 인식할 수 없음.wav' or num_re.search(file_name) !=None:
          file_path = os.path.join(self.wav_dir, file_name)
          os.remove(file_path)
    if self.choice == False:

      for i,file_name in enumerate(os.listdir(self.input_folder)):
        os.mkdir(f'{self.wav_dir}_{i}')
        file_path = os.path.join(self.input_folder,file_name)
        if os.path.isdir(file_path) == True:
          continue;

        split_audio_by_silence(file_path, f'{self.wav_dir}_{i}', self.silence_threshold, self.min_silence_duration)
        make_dataset(f'{self.wav_dir}_{i}')

        for file_error in os.listdir(f'{self.wav_dir}_{i}'):

          if file_error == '음성을 인식할 수 없음.wav' or num_re.search(file_error) !=None:
            file_path = os.path.join(f'{self.wav_dir}_{i}', file_error)
            os.remove(file_path)


#Data Generate

In [ ]:
shutil.rmtree('/content/wav_folder_0')

In [ ]:
silence_threshold = -35
min_silence_duration = 500 #-35 200,300

generator = dataset_generate('/content/drive/MyDrive/wav_mp3_group/wav_mp3/Mint_1.mp3'
                                        , '/content/drive/MyDrive/wav_mp3_group/wav_mp3_2'
                                    ,'wav_folder',silence_threshold,min_silence_duration,False)
generator.call()



chunk 생성 끝
recognize 끝
chunk 생성 끝


KeyboardInterrupt: ignored

#Ipd Audio Test

In [ ]:
ipd.Audio('/content/wav_folder/토요일 다섯 번째 여행.wav',rate = 16000)